In [1]:
import numpy as np
import gensim
from gensim import corpora, models, similarities
from gensim.models.doc2vec import Doc2Vec, LabeledSentence
from sklearn import metrics
from sklearn.cluster import KMeans
from pprint import pprint
import time
import jieba
import jieba.analyse
import pandas as pd
from sklearn.manifold import TSNE
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
def load_stopword():
    #stopword_path = 'F:\\编程练习\\Jupyter notebook\\文本文件夹\\博客爬虫分析文章\\停用词表.txt'
    #stopword_path = 'F:\\学习用夹\\大三下学期\\综合课程设计\\实验数据\\停用词库.txt'
    stopword_path = 'F:\\学习用夹\\大三下学期\\综合课程设计\\实验数据\\scu_stopwords.txt'
    f_stop = open(stopword_path,'r',encoding='UTF-8')
    sw = [line.strip() for line in f_stop]
    f_stop.close()
    stopword_extend = ['\n','湖北','武汉','病毒','中国',
                       '兰州','甘肃','我国','北京','微博','正文',
                       '收起','肖战','消费','复工','复产','全文','消费者','疫情','网页','链接','美国','浙江','杭州','新冠','市场']
    sw.extend(stopword_extend)
    return sw


def jieba_text(data_path):
    stopwords = load_stopword()
    df = pd.read_excel(data_path)
    #df = pd.read_csv(data_path)
    text_seg_list = []
    text_id_list = []
    for index,row in df.iterrows():
        fileId = row['id']
        fileContent = row['内容']
        segs = jieba.analyse.textrank(fileContent,topK=20,withWeight=False,allowPOS=('ns','n','nr','nt','vn')) 
        segments_list = []
        for seg in segs:
            if seg not in stopwords and len(seg) > 1:
                segments_list.append(seg)
        text_seg_list.append(segments_list)
        text_id_list.append(fileId)
        text_seg_list_dic = {'id':text_id_list, 'text':text_seg_list}
    return text_seg_list_dic

    
#导出csv文件
def dataToCsv(file,df):
    file_data = df
    file_data.to_csv(file,index=False)
    print('csv文件已生成在：{}'.format(file))

In [12]:
def LDA(text_seg_list_dic,topic_num):
    text_seg_list = text_seg_list_dic['text']
    # 建立字典
    dictionary = corpora.Dictionary(text_seg_list)
    V = len(dictionary)

    # 转换文本数据为索引，并计数
    corpus = [dictionary.doc2bow(text) for text in text_seg_list]

    # 计算tf-idf值
    corpus_tfidf = models.TfidfModel(corpus)[corpus]

    # 训练模型
    lda = models.LdaModel(corpus_tfidf, num_topics=topic_num, id2word=dictionary)
    #alpha=0.01, eta=0.01, minimum_probability=0.001,update_every=1, chunksize=100, passes=1
    Perplexity = lda.log_perplexity(corpus_tfidf)

    num_show_term = 10  # 每个主题显示几个词
    #print('结果：每个主题的词分布：')
    lda_topic = {'1':[],'2':[],'3':[],'4':[],'5':[],'6':[],'7':[],'8':[],'9':[],'10':[]}
    lda_topic_prob = []
    for topic_id in range(topic_num):
        #print('主题#%d：\t' % topic_id)
        term_distribute_all = lda.get_topic_terms(topicid=topic_id)
        term_distribute = term_distribute_all[:num_show_term]
        term_distribute = np.array(term_distribute)
        term_id = term_distribute[:, 0].astype(np.int)
        #print('词：\t', )
        i = 0
        for t in term_id:
            i += 1
            #print(dictionary.id2token[t], )
            lda_topic[str(i)].append(dictionary.id2token[t])
        #print('\n概率：\t', term_distribute[:, 1])
        lda_topic_prob.append(term_distribute[:, 1])
    #print(lda.print_topics(5))
    
    #lda可视化
    vis_data = pyLDAvis.gensim.prepare(lda,corpus_tfidf,dictionary)
    #pyLDAvis.show(vis_data,open_browser = False)
    
    #输出每个list所属的类别
    topic_list=[]
    for topics in lda.get_document_topics(corpus)[:]:
        #print(topics)
        listj=[]
        for j in topics:
            listj.append(j[1])
        bz=listj.index(max(listj))
        topic_list.append(topics[bz][0])
    corpus_topic_df = pd.DataFrame({'id':text_seg_list_dic['id'], 'topic':topic_list, 'content':text_seg_list})
    dataToCsv('F:\\学习用夹\\大三下学期\\综合课程设计\\实验数据\\LDA结果\\LDA文本分类.csv',corpus_topic_df)
    
    return lda_topic,lda_topic_prob,Perplexity

In [7]:
data_path = 'F:\\学习用夹\\大三下学期\\综合课程设计\\实验数据\\微博消费内容数据01-01_06-20.xlsx'
data_path_1 = 'F:\\学习用夹\\大三下学期\\综合课程设计\\实验数据\\消费_实验集.xlsx'

text_dic = jieba_text(data_path_1)

In [13]:
lda_topic,lda_topic_prob,Perplexity = LDA(text_dic,10)    #第二个系数为lda主题个数
df_topic = pd.DataFrame(lda_topic)
df_topic_prob = pd.DataFrame(lda_topic_prob,columns=['1','2','3','4','5','6','7','8','9','10'])
#print(df_topic)
#print(df_topic_prob)

C:\Users\Mu_Xiaobai\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


csv文件已生成在：F:\学习用夹\大三下学期\综合课程设计\实验数据\LDA结果\LDA文本分类.csv
